In [1]:
import requests
baseUrl = "cli-test1.sandbox.operations.dynamics.com"
url = "https://login.windows.net/Ids-borjomi.com/oauth2/token"

payload=f'grant_type=client_credentials&client_id=05b9c07c-b4fc-4e36-8df3-70b9560a1ce1&client_secret=07sF67Q7WU4h~ce6ZXuPo0_0r5s-1hIlO_&resource=https%3A%2F%2F{baseUrl}'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'login.windows.net',
  'Cookie': 'fpc=ArbVVDqcJvRJq1kuOupppVR98SzvAQAAADIcMNgOAAAA; stsservicecookie=estsfd; x-ms-gateway-slice=estsfd'
}

In [2]:
response = requests.request("POST", url, headers=headers, data=payload)
token = response.json()['access_token']
print("Token OK\r\n")

Token OK



In [3]:
payload = "\r\n"
headers = {
    'OData-Version': '4.0',
    'OData-MaxVersion': '4.0',
    'Content-Type': 'application/json;odata.metadata=full',
    'Accept': 'application/json;odata.metadata=minimal',
    'Accept-Charset': 'UTF-8',
    'Authorization': 'Bearer ' + token,
    'Host': 'cli-test1.sandbox.operations.dynamics.com'
}

In [4]:
url = f"https://{baseUrl}/data/RouteVersionsV2?$select=ItemNumber,ProductColorId,ValidFromDate,ValidToDate,RouteId&$count=true"

In [5]:
response = requests.request("GET", url, headers=headers, data=payload)

In [6]:
records = response.json()["@odata.count"]
print(f"Total {records} records found \r\n")
print("|ItemNumber|ProductColorId|ValidFromDate|ValidToDate|RouteId|MachineName|Performance|ProductName|BomUnitId|")

Total 11 records found 

|ItemNumber|ProductColorId|ValidFromDate|ValidToDate|RouteId|MachineName|Performance|ProductName|BomUnitId|


In [7]:
jsonArrayValue = response.json()["value"]

In [10]:
for element in jsonArrayValue:
    itemNumber      = element["ItemNumber"]
    productColorId  = element["ProductColorId"]
    validFromDate   = element["ValidFromDate"]
    validToDate     = element["ValidToDate"]
    routeId         = element["RouteId"]
    # detail for item
    # get name for selected language
    url = f"https://{baseUrl}/data/RouteOperationsV2?$top=1&$filter=RouteId eq '{routeId}'"

    response = requests.request("GET", url, headers=headers, data=payload)
    machineName = ""
    performance = ""
    routeValue = response.json()["value"]
    for routeItem in routeValue:
        machineName = routeItem["OperationId"] # new field here
        performance = routeItem["OperationNumber"] # new field here
    
    # attribute request
    url = f"https://{baseUrl}/data/ReleasedProductVariantsV2?$select=ProductName&$filter=ItemNumber eq '{itemNumber}' and ProductColorId eq '{productColorId}'&$expand=ReleasedProductMasterV2($select=BOMUnitSymbol)"

    response = requests.request("GET", url, headers=headers, data=payload)

    productName = ""
    bomUnitId   = ""
    addValue = response.json()["value"]
    for addItem in addValue:
        productName     = addItem["ProductName"]
        releasedInfo    = addItem["ReleasedProductMasterV2"]
        bomUnitId       = releasedInfo["BOMUnitSymbol"]
    
    
    print(f"|{itemNumber}|{productColorId}|{validFromDate}|{validToDate}|{routeId}|{machineName}|{performance}|{productName}|{bomUnitId}|")
print("done")


|000001166||1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000001177|DEF|10|||
|000001166||1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000001413|DEF1|10|||
|000002255||2021-01-01T12:00:00Z|1900-01-01T12:00:00Z|000002604|DEF|10|BJ0.50CP11x12UA-ShrinkStd|pcs|
|000003975|AZ|1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000004753|CB|10|BORJOMI 0.50 Glass C01 12P|pcs|
|000003975||1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000005024|101|10|||
|000001166||1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000008014|DEF|10|||
|000001168||1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000010099|CB|10|||
|000011561||2021-03-01T12:00:00Z|1900-01-01T12:00:00Z|000011564|DEF|10|||
|000011561||2021-03-01T12:00:00Z|1900-01-01T12:00:00Z|000011565|DEF|10|||
|000007155||1900-01-01T12:00:00Z|1900-01-01T12:00:00Z|000012004|101|10|||
|BJ0.33Gx12-STD|AZ|2021-04-01T12:00:00Z|1900-01-01T12:00:00Z|000005024|101|10|BJ0.33Gx12-STD-AZ|pcs|
done
